<a href="https://colab.research.google.com/github/ashwinamrutphale/aml-homeworks/blob/master/homework-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')
import pickle
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.image as mpimg
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
import pandas as pd
import scipy as sp
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
activities = ['Brush_teeth','Drink_glass','Climb_stairs','Pour_water',
          'Use_telephone','Eat_meat','Walk','Comb_hair','Eat_soup',
          'Descend_stairs','Getup_bed','Sitdown_chair','Liedown_bed','Standup_chair']

def compute_kmeans(k,df_input):
  return KMeans(n_clusters=k, random_state=0).fit(df_input)

def compute_feature_vector(ds,kmeans):
  return np.unique(kmeans.predict(df),return_counts=True)[1]


def train_and_predict(train,test,k):
  clf = RandomForestClassifier(n_estimator=10,max_depth=4,random_state=0)
  features = train.columns[:k]
  y = pd.factorize(train['activity'])[0]
  clf.fit(train[features],y)
  clf.predict(test[k])
  
  
  


In [0]:
df = pd.read_csv('/content/gdrive/''My Drive''/aml-homeworks/homework5/Accelerometer-2011-03-24-10-24-39-climb_stairs-f1.txt',delim_whitespace=True)


In [23]:
f = compute_feature_vector(df,compute_kmeans(5,df))
f
#np.unique(f, return_counts=True)[1]

array([56, 39, 45, 54, 75])

In [30]:
len(activities)

14